In [ ]:
import numpy as np
from scipy.linalg import sqrtm
from qiskit_ibm_runtime import QiskitRuntimeService
service = QiskitRuntimeService(channel="ibm_quantum")

In [ ]:
# the bases we need to measure the state in, 81=3^4 in total
# the bases go from XXXX to ZZZZ in lexicographical order
bases = []
for i in range(3):
    for j in range(3):
        for k in range(3):
            for l in range(3):
                bases.append([i, j, k, l])

In [ ]:
# layouts that achieve a 9 CNOT circuit on the 7 qubit topology
layouts_7qubit = [
    [1,3,0,5], 
    [3,5,1,4],
    [1,3,2,5],
    [3,5,1,6],
    [3,1,5,0],
    [5,3,4,1],
    [3,1,5,2],
    [5,3,6,1]
          ]

In [ ]:
# layouts that achieve a 9 CNOT circuit on the 5 qubit linear topology
layouts_5qubit_linear = [
    [1,2,0,3],
    [2,1,3,0],
    [2,3,1,4],
    [3,2,4,1]
]

In [ ]:
# layouts that achieve a 9 CNOT circuit on the 5 qubit T-shape topology
layouts_5qubit_Tshape = [
    [1,3,0,4],
    [3,1,4,0],
    [1,3,2,4],
    [3,1,4,2]
]

In [ ]:
# definition of basis measurement gates and pauli basis gates, needed for density matrix reconstruction
I = np.array([[1, 0], [0, 1]])
H = (1/np.sqrt(2))*np.array([[1, 1], [1, -1]])
HSi = (1/np.sqrt(2))*np.array([[1, -1j], [1, 1j]])
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])
# ideal Cabello state
state = (1/np.sqrt(3))*np.array([0, 0, 0, 1, 0, -0.5, -0.5, 0, 0, -0.5, -0.5, 0, 1, 0, 0, 0])
# list basis gates for tomography measurement and pauli basis gates, 
tomo_gates = [H, HSi, I, I]
pauli_gates = [X, Y, Z, I]

In [ ]:
# extra bases, the bases containing the identity operator. we don't need to measure these
# basis states here are of the form [i, j, k, l] with i, j, k, l going from 0 to 3, 
# 0 = X, 1 = Y, 2 = Z and 3 = I
# and at least one of i, j, k, or l is 3
extra_bases = []
extra_bases.append([3, 3, 3, 3])
for i in range(3):
    extra_bases.append([i, 3, 3, 3])
    extra_bases.append([3, i, 3, 3])
    extra_bases.append([3, 3, i, 3])
    extra_bases.append([3, 3, 3, i])
    for j in range(3):
        extra_bases.append([3, 3, i, j])
        extra_bases.append([3, i, 3, j])
        extra_bases.append([3, i, j, 3])
        extra_bases.append([i, 3, 3, j])
        extra_bases.append([i, 3, j, 3])
        extra_bases.append([i, j, 3, 3])
        for k in range(3):
                extra_bases.append([i, j, k, 3])
                extra_bases.append([i, j, 3, k])
                extra_bases.append([i, 3, j, k])
                extra_bases.append([3, i, j, k])

In [ ]:
def classical_fidelity(basis, num_run):
    '''calculates the classical fidelity of the measured result
    with respect to the ideal cabello state in the chosen basis'''
    gates = [H, HSi, I]
    gate = np.kron(np.kron(np.kron(gates[basis[0]], gates[basis[1]]), gates[basis[2]]), gates[basis[3]])
    final_state = gate.dot(state)
    perf_probs = {f'{k:04b}': abs(final_state[k])**2 for k in range(16)}
    meas_probs = {f'{bitstring:04b}'[::-1]: prob for (bitstring, prob) in res.quasi_dists[num_run].items()}
    fidelity = 0
    for bitstring in perf_probs:
        fidelity += np.sqrt(perf_probs[bitstring]*meas_probs.get(bitstring, 0))
    return fidelity**2

In [ ]:
def mitigate(probs, p01s, p10s):
    '''create readout error mitigated probability distribution
    from raw measurement probability distribution and tensor product readout error parameters
    parameters:
    ----------
    probs: unmitigated probability distribution
    p01s: p(0->1) readout errors in order for the relevant qubits
    p10s: p(1->0) readout errors in order for the relevant qubits
    
    output:
    error mitigated probability distribution.
    '''
    ps = list(probs.values())
    As = []
    for p01, p10 in zip(p01s, p10s):
        As.append([[1-p01, p10], [p01, 1-p10]])
    T = np.kron(As[0], As[1])
    for A in As[2:]:
        T = np.kron(T, A)
    mitig_probs = np.linalg.inv(T).dot(ps)
    return {key: mitig_probs[i] for i, key in enumerate(probs.keys())}

In [ ]:
def classical_fidelity_mitig(basis, num_run):
    '''calculates the classical fidelity of the error mitigated result
    with respect to the ideal cabello state in the chosen basis'''
    gates = [H, HSi, I]
    gate = np.kron(np.kron(np.kron(gates[basis[0]], gates[basis[1]]), gates[basis[2]]), gates[basis[3]])
    final_state = gate.dot(state)
    perf_probs = {f'{k:04b}': abs(final_state[k])**2 for k in range(16)}
    meas_probs = {f'{bitstring:04b}'[::-1]: prob for (bitstring, prob) in res.quasi_dists[num_run].items()}
    mitig_probs = mitigate(meas_probs, p01s, p10s)
    fidelity = 0
    for bitstring in perf_probs:
        fidelity += np.sqrt(perf_probs[bitstring]*mitig_probs.get(bitstring, 0))
    return fidelity**2

In [ ]:
# calculates the p(1->0) readout error from the error circuit measurements for the chosen qubit
def calc_p10(prob_dist, qubit):
    p = 0
    for (bs, prob) in prob_dist.items():
        if f'{bs:07b}'[::-1][qubit] == '0':
            p += prob
    return p

In [ ]:
# calculates the p(0->1) readout error from the error circuit measurements for the chosen qubit
def calc_p01(prob_dists, qubit):
    p = 0
    for prob_dist in prob_dists:
        for (bs, prob) in prob_dist.items():
            if f'{bs:07b}'[::-1][qubit] == '1':
                p += prob
    return p / len(prob_dists)

In [ ]:
def density_matrix(res):
    '''
    Reconstructing the density matrix from the measurement results.
    The equation used (\sigma are the four Pauli matrices): 
    \rho = (1/16)*sum_{ijkl=0}^{3} {\sigma_i \sigma_j \sigma_k \sigma_l \Tr{\sigma_i \sigma_j \sigma_k \sigma_l \rho}},
    where \Tr{\sigma_i \sigma_j \sigma_k \sigma_l \rho} is estimated from the measurement results.
    parameters:
    ----------
    res: the qiskit SamplerResult object
    
    output: 
    the reconstructed density matrix.
    '''
    rho = np.zeros((16, 16), dtype='complex128')
    for (basis, num_run) in zip(bases, range(len(bases))):
        meas_probs = {f'{bitstring:04b}'[::-1]: prob for (bitstring, prob) in res.quasi_dists[num_run].items()}
        pauli_matrix = np.kron(np.kron(np.kron(pauli_gates[basis[0]], pauli_gates[basis[1]]),
                                       pauli_gates[basis[2]]), pauli_gates[basis[3]])
        trace_rho = sum((-1)**(state.count('1') % 2)*meas_probs[state] for state in meas_probs)
        rho += trace_rho * pauli_matrix
    for basis in extra_bases:
        og_basis = [b if b!=3 else 2 for b in basis]
        no = og_basis[0]*27+og_basis[1]*9+og_basis[2]*3+og_basis[3]
        meas_probs = {f'{bitstring:04b}'[::-1]: prob for (bitstring, prob) in res.quasi_dists[no].items()}
        pauli_matrix = np.kron(np.kron(np.kron(pauli_gates[basis[0]], pauli_gates[basis[1]]),
                                       pauli_gates[basis[2]]), pauli_gates[basis[3]])
        trace_rho = 0
        for curr_state in meas_probs:
            new_state = curr_state
            for i in range(len(basis)):
                if basis[i] == 3:
                    new_state = new_state[:i] + '0' + new_state[i+1:]
            trace_rho += (-1)**(new_state.count('1') % 2)*meas_probs[curr_state]
        rho += trace_rho * pauli_matrix
    return (1/16)*rho

In [ ]:
def density_matrix_mitig(res, p01s, p10s):
    '''
    Reconstructing the density matrix from the measurement results.
    The equation used (\sigma are the four Pauli matrices): 
    \rho = (1/16)*sum_{ijkl=0}^{3} {\sigma_i \sigma_j \sigma_k \sigma_l \Tr{\sigma_i \sigma_j \sigma_k \sigma_l \rho}},
    where \Tr{\sigma_i \sigma_j \sigma_k \sigma_l \rho} is estimated from the measurement results
    and mitigated using simple inversion assuming a tensor product matrix readout error noise model.
    parameters:
    ----------
    res: the qiskit SamplerResult object
    p01s: p(0->1) readout errors in order for the relevant qubits
    p10s: p(1->0) readout errors in order for the relevant qubits
    
    output:
    the reconstructed density matrix.
    '''
    rho = np.zeros((16, 16), dtype='complex128')
    for (basis, num_run) in zip(bases, range(len(bases))):
        meas_probs = {f'{bitstring:04b}'[::-1]: prob for (bitstring, prob) in res.quasi_dists[num_run].items()}
        mitig_probs = mitigate(meas_probs, p01s, p10s)
        pauli_matrix = np.kron(np.kron(np.kron(pauli_gates[basis[0]], pauli_gates[basis[1]]),
                                       pauli_gates[basis[2]]), pauli_gates[basis[3]])
        trace_rho = sum((-1)**(state.count('1') % 2)*mitig_probs[state] for state in mitig_probs)
        rho += trace_rho * pauli_matrix
    for basis in extra_bases:
        og_basis = [b if b!=3 else 2 for b in basis]
        no = og_basis[0]*27+og_basis[1]*9+og_basis[2]*3+og_basis[3]
        meas_probs = {f'{bitstring:04b}'[::-1]: prob for (bitstring, prob) in res.quasi_dists[no].items()}
        mitig_probs = mitigate(meas_probs, p01s, p10s)
        pauli_matrix = np.kron(np.kron(np.kron(pauli_gates[basis[0]], pauli_gates[basis[1]]),
                                       pauli_gates[basis[2]]), pauli_gates[basis[3]])
        trace_rho = 0
        for curr_state in mitig_probs:
            new_state = curr_state
            for i in range(len(basis)):
                if basis[i] == 3:
                    new_state = new_state[:i] + '0' + new_state[i+1:]
            trace_rho += (-1)**(new_state.count('1') % 2)*mitig_probs[curr_state]
        rho += trace_rho * pauli_matrix
    return (1/16)*rho

In [ ]:
def quantum_fidelity(rho1, rho2):
    '''Function to calculate the quantum fidelity between two density matrices rho1 and rho2'''
    return (np.trace(sqrtm(np.matmul(np.matmul(sqrtm(rho1), rho2), sqrtm(rho1)))))**2

In [ ]:
def calculate_QST_quantum_fidelity(job_id, layout):
    '''calculates raw and mitigated quantum fidelity of Cabello state tomography measurement result.
    parameters:
    ----------
    job_id: id of the qiskit job to evaluate
    layout_num: layout of the qubits used in the job
    
    output:
    quantum fidelity of the unmitigated and the mitigated state tomography measurement result'''
    job = service.job(job_id)
    res = job.result()
    n = job.backend().configuration().n_qubits
    if n == 7:
        layouts = layouts_7qubit
    if n == 5:
        layouts = layouts_5qubit
    p01s = []
    p10s = []
    readout_meas = res.quasi_dists[81:]
    for i, qubit in enumerate(layout):
        p01s.append(calc_p01(readout_meas[:i]+readout_meas[i+1:], qubit))
        p10s.append(calc_p10(readout_meas[i], qubit))
    rho_meas = density_matrix(res) # unmitigated density matrix
    rho_mitig = density_matrix_mitig(res, p01s, p10s) # mitigated density matrix
    rho_ideal = np.outer(state, state)
    return quantum_fidelity(rho_meas, rho_ideal).real, quantum_fidelity(rho_mitig, rho_ideal).real